In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
import os, librosa

In [ ]:
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

# 음성정보 배열로 처리
def load_wav(wav_path):
  wav, _ = librosa.load(wav_path, sr=8000)
  return wav

# 스크립트별로 전처리
def preprocess_audio(csv):
  
  df = pd.read_csv(csv)
  seg_id = list(df['Segment ID'][1:])
  
  # 모아야 할 정보!
  speaker = []
  audio = []
  '''
  emotion = df['Total Evaluation'][1:]
  sentiment = []
  
  sentidict = {'positive': ["joy"], 'negative': ["anger", "disgust", "fear", "sadness"], 'neutral': ["neutral", "surprise"]}
  
  #센티먼트 담기
  for emo in emotion:
    for sent, emo_list in sentidict.items():
       if emo in emo_list:
          sentiment.append(sent)
  '''

  # 오디오 담기(csv 파일에 있는 seg id 순서대로)
  for segment in tqdm(seg_id):
    wav = load_wav('audio/'+segment+'.wav')  
    audio.append(wav)
  
  
  # 스크립트 구분(세션번호 + 스크립트번호), 화자 이름 담기
  script_id = []

  for id in seg_id:
    id = id.split('_')
    script_id.append(id[0] + '_' + id[1])
    speaker.append(id[2])
  
  '''
  # 스크립트별로 담을 때!
  script_set = sorted(list(set(script_id)))

  speaker_dict = {id:[] for id in script_set}
  audio_dict = {id:np.array([]).astype(np.float32) for id in script_set}
  emotion_dict = {id:[] for id in script_set}
  sentiment_dict = {id:[] for id in script_set}
  
 
  for id, spk in zip(script_id, speaker):
    speaker_dict[id].append(spk)
    
  for id, wav in zip(script_id, audio):
    audio_dict[id] = np.concatenate((audio_dict[id], wav))
    
  for id, emo in zip(script_id, emotion):
    emotion_dict[id].append(emo)

  for id, snt in zip(script_id, sentiment):
    sentiment_dict[id].append(snt)
    
  return script_set, speaker_dict, audio_dict, emotion_dict, sentiment_dict
  '''
  return seg_id, script_id, speaker, audio  # 리스트 출력, 정확히 담화 순서대로 담겨있음
'''
#스크립트별로 얻을 때
script_set, speaker_dict, audio_dict, emotion_dict, sentiment_dict = process_by_script('Sess01_eval.csv')

print(script_set)
print(speaker_dict)
print(audio_dict)
print(emotion_dict)
print(sentiment_dict)
'''

"\n#스크립트별로 얻을 때\nscript_set, speaker_dict, audio_dict, emotion_dict, sentiment_dict = process_by_script('Sess01_eval.csv')\n\nprint(script_set)\nprint(speaker_dict)\nprint(audio_dict)\nprint(emotion_dict)\nprint(sentiment_dict)\n"

In [ ]:
total_seg_id = []
total_script_id = []
total_speaker = []
total_audio = []

In [ ]:
csv_list = sorted(os.listdir('annotation'))

for csv_path in csv_list:
  seg_id, script_id, speaker, audio = preprocess_audio('annotation/'+csv_path)
  total_seg_id += seg_id 
  total_script_id += script_id
  total_speaker += speaker 
  total_audio += audio

print(len(total_seg_id), len(total_script_id), len(total_speaker), len(total_audio))

 25%|██▌       | 83/329 [00:52<02:34,  1.59it/s]

In [ ]:
!pip install transformers

In [ ]:
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torch

In [ ]:
# 오디오 데이터를 텐서로 변환
audio_ds = []
for wav in tqdm(total_audio):
  w = torch.FloatTensor(wav)
  audio_ds.append(w)

In [ ]:
# pretrained Wav2Vec2 불러오기
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
encoder = Wav2Vec2Model.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

In [ ]:
# wav data 배열 모델에 넣어서 인코딩
def encoding(wav_arr, processor = None, encoder = None, return_hidden_state=False):
    
    assert bool(processor) == bool(encoder)
    
    inputs = processor(wav_arr,
                       sampling_rate=16000,
                       return_attention_mask=True,
                       return_tensors="pt")
    
    Inputs = inputs.to('cuda')
    Encoder = encoder.to('cuda')
    outputs = encoder(output_hidden_states=return_hidden_state, **inputs)

    return outputs

In [ ]:
# hidden, feature vector 추출

def extract_vector(data):
  with torch.no_grad():
    encoded = encoding(data, processor = processor, encoder = encoder, return_hidden_state=True)  # encoding 함수 사용

    hidden_vec = encoded.last_hidden_state.mean(dim=1).tolist()                                                         
    feature_vec = encoded.extract_features.mean(dim=1).tolist()

    return hidden_vec, feature_vec

In [ ]:
# hidden, feature vector 뽑기
total_hidden_vector = []
total_feature_vector = []

for wav in tqdm(audio_ds):
  hidden, feature = extract_vector(wav)
  total_hidden_vector.append(hidden)
  total_feature_vector.append(feature)

In [ ]:
print(len(total_hidden_vector), len(total_feature_vector))

In [ ]:
# 항목: seg_id, script_id, speaker, hidden_vector, feature_vector 
# 같은 인덱스 = 같은 발화 정보

total_dataset = {'seg_ID': total_seg_id,
                 'script_id': total_script_id,
                 'speaker': total_speaker,
                 'hidden_vector': total_hidden_vector,
                 'feature_vector': total_feature_vector}      

In [ ]:
total_dataset = []


In [ ]:
#json 파일로 내보내기
import json
with open('total_dataset.json', 'w') as f : 
	json.dump(total_dataset, f)